In [5]:
config_excel_path="abfss://enterpriseanalytics@fedsaentanalytics.dfs.core.windows.net/config/wm_ingest_config.xlsx"

In [9]:
import pandas as pd

config_path="abfss://enterpriseanalytics@fedsaentanalytics.dfs.core.windows.net/config/fe_config_ingest/"

df = pd.read_excel(config_excel_path)
source_df = spark.createDataFrame(df)
source_df.createOrReplaceTempView("tmp_config")

display(source_df)

In [10]:
# do checks
configs = spark.sql("""select lower(array_join(collect_list(distinct concat('\\'',config_name,'\\'')), ',')) configs
 from tmp_config
""").first().configs

print(configs)

if spark.sql("select az_id from tmp_config group by az_id having count(*) > 1").count() > 0:
    raise Exception(f"There are duplicate az_ids in {config_excel_path}")

if spark.sql("select * from tmp_config").count() <= 0:
    raise Exception(f"There are no rows returned from {config_excel_path}")

if (spark.sql("select * from tmp_config").count()) != (spark.sql(f"select * from tmp_config where config_name in ({configs})").count()):
    raise Exception(f"There are issues with count returned when filtering on distinct config_names from {config_excel_path}")

In [12]:
spark.sql(f"delete from delta.`{config_path}` t where lower(config_name) in ({configs})")

spark.sql(f"""
insert into delta.`{config_path}` (uuid, config_name, src_type, src_db_name, src_schema_name, src_table_name, src_col_def, src_query_init, src_query_inc, src_conn_secret_name, dest_delta_storage_account, dest_delta_table_container, temp_stage_container, mode, inc_del_filter, primary_key, partition_by, tag, src_file_container, src_file_directory, src_file_name, excel_sheet_name, has_header, rows_to_skip, is_disabled, dw_server_name, dw_db_name, dw_schema_name, dw_table_name)
    select uuid() uuid, config_name, src_type, src_db_name, src_schema_name, src_table_name, src_col_def, src_query_init, src_query_inc, src_conn_secret_name, dest_delta_storage_account, dest_delta_table_container, temp_stage_container, mode, inc_del_filter, primary_key, partition_by, tag, src_file_container, src_file_directory, src_file_name, excel_sheet_name, has_header, rows_to_skip, is_disabled, dw_server_name, dw_db_name, dw_schema_name, dw_table_name
    from tmp_config
""")

In [7]:
display(spark.sql(f"select * from delta.`{config_path}` where config_name in ({configs})"))
display(spark.sql(f"DESCRIBE HISTORY delta.`{config_path}`"))

In [ ]:
from delta.tables import *

deltaTableConfig = DeltaTable.forPath(spark, config_path)
deltaTableConfig.optimize().executeCompaction()

deltaTableConfigPart = DeltaTable.forPath(spark, config_path.replace("/fe_config_ingest/", "/fe_config_ingest_part/"))
deltaTableConfigPart.optimize().executeCompaction()